In [ ]:
import numpy as np

test_data = np.load("/mnt/nas05/data01/francesco/sdo_img2img/sde_mag2mag_v2/progetto_simone/data/pickled_maps/map_2024_1_1_0_0_0.npy", allow_pickle=True)

In [ ]:
test_data

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(test_data[0])

In [ ]:
test_data[0].shape

In [ ]:
np.std(test_data[0])

In [ ]:
import pandas as pd

stats_df = pd.read_csv(r"/mnt/nas05/data01/francesco/sdo_img2img/sde_mag2mag_v2/npy_metrics.csv")

In [ ]:
stats_df['filename'].iloc[0]

In [ ]:
stats_df["timestamp_str"] = stats_df["filename"].str.extract(r"map_(\d+_\d+_\d+_\d+_\d+_\d+)\.npy")


In [ ]:
stats_df["datetime"] = pd.to_datetime(stats_df["timestamp_str"], format="%Y_%m_%d_%H_%M_%S")


In [ ]:
stats_df = stats_df.sort_values(by="datetime", ascending=True)

In [ ]:
(stats_df['mean']).mean(), stats_df['std'].mean()  # or better: use RMS of stds


stats_df

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

stats = ['std', 'max', 'min', 'mean']
# Plot
for value in stats:
    plt.figure(figsize=(12, 5))
    plt.plot(stats_df['datetime'], stats_df[f'{value}'], color='black')

    # Labels and formatting
    plt.xlabel("Year", fontsize=25)
    plt.ylabel(f"{value}", fontsize=25)
    plt.title(f"{value}", fontsize=25)
    plt.xticks(rotation=45, fontsize=15)
    plt.yticks(fontsize=15)
    plt.grid(True)
    plt.show()
    plt.tight_layout()
    plt.savefig(f"/mnt/nas05/data01/francesco/sdo_img2img/sde_mag2mag_v2/plots/{value}.png")
    plt.close()


In [1]:
import sys
sys.path.append("/mnt/nas05/data01/francesco/progetto_simone/ionosphere")  # add current dir, where src/ is located

from src.data.dataset import IonoDataset

WARNING[XFORMERS]: xFormers can't load C++/CUDA extensions. xFormers was built for:
    PyTorch 2.6.0+cu124 with CUDA 1204 (you have 2.6.0+cu118)
    Python  3.10.16 (you have 3.10.12)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details
/mnt/nas05/data01/francesco/progetto_simone/ionosphere/src/vit.py:26: FutureWarning: xformers.components is deprecated and is not maintained anymore. It might be removed in a future version of xFormers 
  import xformers.components.attention.core as xfa


In [ ]:
train_dataset = IonoDataset(
    path="/mnt/nas05/data01/francesco/sdo_img2img/sde_mag2mag_v2/progetto_simone/data/pickled_maps",
    transforms=True,
    split='train',
    seed=42
)

In [ ]:
train_dataset[0]

In [ ]:
import torch
torch.max(train_dataset[0][0]), torch.min(train_dataset[0][0]), torch.mean(train_dataset[0][0]), torch.std(train_dataset[0][0])

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(train_dataset[0][0][0].cpu().numpy())
plt.colorbar()

In [ ]:
from src.data.dataset import get_data_objects

train_dataset, train_sampler, train_data_loader = get_data_objects(
    path="/mnt/nas05/data01/francesco/sdo_img2img/sde_mag2mag_v2/progetto_simone/data/pickled_maps",
    batch_size=4,
    distributed=False,
    num_data_workers=1,
    split='train',
    seed=42
)

In [ ]:
train_dataset[0]

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(train_dataset[0][0][0].cpu().numpy())
plt.colorbar()

In [2]:
import pandas as pd

df = pd.read_csv("/mnt/nas05/data01/francesco/sdo_img2img/sde_mag2mag_v2/progetto_simone/data/params.csv")

In [3]:
df

,filename,float1,float2,float3,float4
0,map_2024_1_1_13_48_0.npy,3.98,-2.89,2.38,-344.7
1,map_2024_1_1_11_22_0.npy,1.53,-8.16,-1.46,-324.2
2,map_2024_1_1_9_6_0.npy,0.00,0.00,0.00,0.0
3,map_2024_1_1_6_50_0.npy,-2.64,-0.16,-3.36,-330.8
4,map_2024_1_1_22_50_0.npy,2.08,-8.40,5.39,-346.2
...,...,...,...,...,...
263147,map_2024_12_31_23_48_0.npy,-3.73,13.90,0.80,-428.2
263148,map_2024_12_31_23_50_0.npy,-3.65,13.17,0.74,-424.0
263149,map_2024_12_31_23_52_0.npy,-3.11,12.52,0.48,-429.8
263150,map_2024_12_31_23_54_0.npy,-8.00,10.61,2.86,-408.2


In [4]:
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

features = df[["float1", "float2", "float3", "float4"]]
scaler = StandardScaler()
X_scaled = scaler.fit_transform(features)

In [ ]:
# cluster parameters
min_cluster = 2
max_cluster = 50
from yellowbrick.cluster import KElbowVisualizer, SilhouetteVisualizer, InterclusterDistance
from sklearn.cluster import KMeans, MeanShift

model = KMeans()
visualizer = KElbowVisualizer(model, k=(min_cluster, max_cluster + 1))
visualizer.fit(X_scaled)  # Fit the data to the visualizer
visualizer.show() 

In [6]:
# with optimal value of the elbo
from sklearn.cluster import KMeans
optimal_k = 13 #visualizer.elbow_value_
kmeans = KMeans(n_clusters=optimal_k, random_state=42)
df["cluster"] = kmeans.fit_predict(X_scaled)

In [7]:
df["cluster"]

0         11
1          3
2          2
3          7
4         11
          ..
263147     9
263148     9
263149     9
263150     9
263151    10
Name: cluster, Length: 263152, dtype: int32

In [9]:
import numpy as np
np.unique(df['cluster'], return_counts=True)

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12], dtype=int32),
 array([ 7637,  5241, 42485, 38962, 37826, 21266,  7207, 22558,   837,
        16980, 21470, 25161, 15522]))

In [ ]:
import matplotlib.pyplot as plt
# from sklearn.manifold import TSNE
from openTSNE import TSNE

# # Reduce dimensionality to 2D for visualization
# tsne = TSNE(n_components=2, random_state=42, perplexity=50, learning_rate=200)
# X_embedded = tsne.fit_transform(X_scaled)
tsne = TSNE(
    n_components=2,
    perplexity=100,
    metric="cosine",
    n_jobs=8,  # number of threads
    random_state=42
)
X_embedded = tsne.fit(X_scaled)

# Add t-SNE coordinates to DataFrame for plotting
df["tsne_1"] = X_embedded[:, 0]
df["tsne_2"] = X_embedded[:, 1]

# Set figure size to match 1920x1080 pixels with dpi=100
fig, ax = plt.subplots(figsize=(19.2, 10.8), dpi=200)

# Plot each cluster
for cluster_id in sorted(df["cluster"].unique()):
    subset = df[df["cluster"] == cluster_id]
    ax.scatter(subset["tsne_1"], subset["tsne_2"], s=10, label=f"Cluster {cluster_id}", alpha=0.7)

# Titles and labels
ax.set_title(f"t-SNE Projection of KMeans Clusters (k={optimal_k})", fontsize=16)
ax.set_xlabel("t-SNE 1", fontsize=12)
ax.set_ylabel("t-SNE 2", fontsize=12)
ax.legend(markerscale=2, fontsize="small", loc="best")
ax.grid(True)

# Save as Full HD PNG
plt.tight_layout()
# plt.savefig("tsne_clusters_fullhd.png", dpi=100)  # or dpi=200 for 4K
plt.show()

